In [1]:
%config Completer.use_jedi = False
import sys
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import time
import threading
import gc
import copy

sys.path.append("/end/home/rh2310/morpho_repo/turing_codebase")
from turing.utils import *
import turing.pinns as tu
from turing.loss_functions import *

GPU = True
import os

if GPU:
    txt_device = 'gpu:0'
else:
    txt_device = 'cpu:0'    
    os.environ["CUDA_VISIBLE_DEVICES"]="-1"

2022-10-14 18:48:26.278739: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-10-14 18:48:32.390186: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-10-14 18:48:32.400605: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:2d:00.0 name: Quadro RTX 6000 computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.65GiB deviceMemoryBandwidth: 625.94GiB/s
2022-10-14 18:48:32.400674: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-10-14 18:48:32.405304: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-10-14 18:48:32.409072: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10

# Load data from the output of Brusselator model

In [4]:
import os
data_path = os.path.abspath("../turing_Brusselator.npy")
with open(data_path, 'rb') as f:
    data = np.load(f)
    
data_path = os.path.abspath("../turing_t_Brusselator.npy")
with open(data_path, 'rb') as f:
    t_star = np.load(f) 
    
# Thining the dataset
t_star = t_star[20::16]
data = data[..., 20::16]

T = t_star.shape[0]    
    
L = 1.0
x_size = data.shape[1]
y_size = data.shape[2]
N = x_size*y_size

model_params = {'training_data_size': T*N,#T*32,
                'pde_data_size': (T*N)//(32),
                'boundary_data_size':((x_size + y_size)*T)//(8)}

dataset = create_dataset(data, t_star, N, T, L, **model_params)
lb = dataset['lb']
ub = dataset['ub']
obs_X = dataset['obs_input']
obs_Y = dataset['obs_output']
pde_X = dataset['pde']


In [3]:
T

15

# Run Koch-Meinhardt's ASDM model with given diffusion constants and correct space-time scalse to find other parameters

In [4]:
#layers = [3, 64, 64, 64, 64, 2]
layers = [3, 128, 128, 128, 128, 2]

pinn = tu.NN(layers, lb, ub, dtype=tf.float64)
pde_loss = ASDM(dtype=tf.float64, init_value=1.0)
pde_non_zero = Non_zero_params("ASDM", parameters=[pde_loss.D_u, pde_loss.D_v], print_precision=".10f")

l2_norm = L2()
l_inf_norm = L_Inf()

In [7]:
pinn.save("/end/home/rh2310/save_tests", "pinn_1")

In [8]:
pde_loss.save("/end/home/rh2310/save_tests", "pde_loss_1")

INFO:tensorflow:Unsupported signature for serialization: ((<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f349c28deb0>, TensorSpec(shape=(1920, 3), dtype=tf.float64, name='x')), {}).
INFO:tensorflow:Unsupported signature for serialization: ((<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f349c28deb0>, TensorSpec(shape=(1920, 3), dtype=tf.float64, name='x')), {}).
INFO:tensorflow:Unsupported signature for serialization: ((<tensorflow.python.framework.func_graph.UnknownArgument object at 0x7f349c28deb0>, TensorSpec(shape=(1920, 3), dtype=tf.float64, name='x')), {}).
INFO:tensorflow:Assets written to: /end/home/rh2310/save_tests/pde_loss_1/assets


In [3]:
pinn2 = tu.NN.restore("/end/home/rh2310/save_tests", "pinn_1")

In [5]:
pinn2.net([[0,0,0]])

2022-10-14 13:00:23.646176: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10


<tf.Tensor: shape=(1, 2), dtype=float64, numpy=array([[ 0.04610581, -0.0373262 ]])>

In [11]:
pinn.net([[0,0,0]])

<tf.Tensor: shape=(1, 2), dtype=float64, numpy=array([[ 0.04610581, -0.0373262 ]])>

In [4]:
pde_loss2 = tu.NN.restore("/end/home/rh2310/save_tests", "pde_loss_1")

In [7]:
tf.saved_model.save(pinn, '/tmp/pinn', signatures=pinn.net.get_concrete_function(
    tf.TensorSpec((None, 3), pinn.dtype)))

INFO:tensorflow:Assets written to: /tmp/pinn/assets


In [4]:
pinn2 = tf.saved_model.load('/tmp/pinn')

In [32]:
pinn2.gradients = pinn.gradients

In [33]:
pinn2.gradients_tape = pinn.gradients_tape

In [34]:
pinn2.dtype = pinn.dtype

In [16]:
f = pde_loss.loss.get_concrete_function(pinn, tf.TensorSpec((None, 3), pinn.dtype))

In [6]:
pinn2.net([[0, 0, 0]])

<tf.Tensor: shape=(1, 2), dtype=float64, numpy=array([[ 0.01824168, -0.10053043]])>

tf.float64

In [7]:
tf.saved_model.save(pde_loss, '/tmp/pde_loss', signatures=pde_loss.loss.get_concrete_function(
    pinn2, x=tf.TensorSpec((None, 3), pinn.dtype)))

NameError: name 'pde_loss' is not defined

In [8]:
bb = tf.saved_model.load('/tmp/pde_loss')

In [9]:
bb.

_SignatureMap({})

In [59]:
pde_loss2 = ASDM2.restote(tf.saved_model.load('/tmp/pde_loss'))

In [58]:
class ASDM2(Loss):
    def __init__(
        self,
        dtype,
        init_value=10.0,
        D_u=None,
        D_v=None,
        sigma_u=None,
        sigma_v=None,
        mu_u=None,
        rho_u=None,
        rho_v=None,
        kappa_u=None,
        print_precision=".5f",
    ):
        """ASDM PDE loss

        if the parameter is None, it becomes traiable with initial value set as init_vale,
        otherwise, it will be a constant
        """

        super().__init__(name="Loss_ASDM", print_precision=print_precision)

        self._trainables_ = ()        
        if D_u is None:
            self.D_u = tf.Variable(
                [init_value], dtype=dtype, name="D_u", constraint=lambda z: tf.clip_by_value(z, 1e-6, 1e10)
            )
            self._trainables_ += (self.D_u,)
        elif isinstance(D_u, tf.Variable):
            self.D_u = D_u
            self._trainables_ += (self.D_u,)
        elif isinstance(D_u, tf.constant):
            self.D_u = D_u
        else:
            self.D_u = tf.constant(D_u, dtype=dtype, name="D_u")

        if D_v is None:
            self.D_v = tf.Variable(
                [init_value], dtype=dtype, name="D_v", constraint=lambda z: tf.clip_by_value(z, 1e-6, 1e10)
            )
            self._trainables_ += (self.D_v,)
        elif isinstance(D_v, tf.Variable):
            self.D_v = D_v
            self._trainables_ += (self.D_v,)
        elif isinstance(D_v, tf.constant):
            self.D_v = D_v
        else:
            self.D_v = tf.constant(D_v, dtype=dtype, name="D_v")

        if sigma_u is None:
            self.sigma_u = tf.Variable(
                [init_value], dtype=dtype, name="sigma_u", constraint=lambda z: tf.clip_by_value(z, 0, 1e10)
            )
            self._trainables_ += (self.sigma_u,)
        elif isinstance(sigma_u, tf.Variable):
            self.sigma_u = sigma_u
            self._trainables_ += (self.sigma_u,)
        elif isinstance(sigma_u, tf.constant):
            self.sigma_u = sigma_u
        else:
            self.sigma_u = tf.constant(sigma_u, dtype=dtype, name="sigma_u")

        if sigma_v is None:
            self.sigma_v = tf.Variable(
                [init_value], dtype=dtype, name="sigma_v", constraint=lambda z: tf.clip_by_value(z, 0, 1e10)
            )
            self._trainables_ += (self.sigma_v,)
        elif isinstance(sigma_u, tf.Variable):
            self.sigma_v = sigma_v
            self._trainables_ += (self.sigma_v,)
            
        elif isinstance(sigma_v, tf.constant):
            self.sigma_v = sigma_v
        else:
            self.sigma_v = tf.constant(sigma_v, dtype=dtype, name="sigma_v")

        if mu_u is None:
            self.mu_u = tf.Variable(
                [init_value], dtype=dtype, name="mu_u", constraint=lambda z: tf.clip_by_value(z, 0, 1e10)
            )
            self._trainables_ += (self.mu_u,)
        elif isinstance(mu_u, tf.Variable):
            self.mu_u = mu_u
            self._trainables_ += (self.mu_u,)
        elif isinstance(mu_u, tf.constant):
            self.mu_u = mu_u
        else:
            self.mu_u = tf.constant(mu_u, dtype=dtype, name="mu_u")

        if rho_u is None:
            self.rho_u = tf.Variable(
                [init_value], dtype=dtype, name="rho_u", constraint=lambda z: tf.clip_by_value(z, 0, 1e10)
            )
            self._trainables_ += (self.rho_u,)
        elif isinstance(rho_u, tf.Variable):
            self.rho_u = rho_u
            self._trainables_ += (self.rho_u,)
        elif isinstance(rho_u, tf.constant):
            self.rho_u = rho_u
        else:
            self.rho_u = tf.constant(rho_u, dtype=dtype, name="rho_u")

        if rho_v is None:
            self.rho_v = tf.Variable(
                [init_value], dtype=dtype, name="rho_v", constraint=lambda z: tf.clip_by_value(z, 0, 1e10)
            )
            self._trainables_ += (self.rho_v,)
        elif isinstance(rho_v, tf.Variable):
            self.rho_v = rho_v
            self._trainables_ += (self.rho_v,)
        elif isinstance(rho_v, tf.constant):
            self.rho_v = rho_v
        else:
            self.rho_v = tf.constant(rho_v, dtype=dtype, name="rho_v")

        if kappa_u is None:
            self.kappa_u = tf.Variable(
                [init_value], dtype=dtype, name="kappa_u", constraint=lambda z: tf.clip_by_value(z, 0, 1e10)
            )
            self._trainables_ += (self.kappa_u,)
        elif isinstance(kappa_u, tf.Variable):
            
            self.kappa_u = kappa_u
            self._trainables_ += (self.kappa_u,)
        elif isinstance(kappa_u, tf.constant):
            self.kappa_u = kappa_u
        else:
            self.kappa_u = tf.constant(kappa_u, dtype=dtype, name="kappa_u")

    @classmethod
    def restote(cls, saved):        
        return cls(None, D_u=saved.D_u,
        D_v=saved.D_v,
        sigma_u=saved.sigma_u,
        sigma_v=saved.sigma_v,
        mu_u=saved.mu_u,
        rho_u=saved.rho_u,
        rho_v=saved.rho_v,
        kappa_u=saved.kappa_u)
    
    @tf.function
    def loss(self, pinn, x):
        outputs = pinn.net(x)
        p1, p2 = pinn.gradients(x, outputs)

        u = outputs[:, 0]
        v = outputs[:, 1]

        # u_x = tf.cast(p1[0][:, 0], pinn.dtype)
        # u_y = tf.cast(p1[0][:, 1], pinn.dtype)
        u_t = tf.cast(p1[0][:, 2], pinn.dtype)

        u_xx = tf.cast(p2[0][:, 0], pinn.dtype)
        u_yy = tf.cast(p2[0][:, 1], pinn.dtype)

        # v_x = tf.cast(p1[1][:, 0], pinn.dtype)
        # v_y = tf.cast(p1[1][:, 1], pinn.dtype)
        v_t = tf.cast(p1[1][:, 2], pinn.dtype)

        v_xx = tf.cast(p2[1][:, 0], pinn.dtype)
        v_yy = tf.cast(p2[1][:, 1], pinn.dtype)

        D_u = self.D_u
        D_v = self.D_v
        sigma_u = self.sigma_u
        sigma_v = self.sigma_v
        mu_u = self.mu_u
        rho_u = self.rho_u
        rho_v = self.rho_v
        kappa_u = self.kappa_u

        f = u * u * v / (1.0 + kappa_u * u * u)
        f_u = u_t - D_u * (u_xx + u_yy) - rho_u * f + mu_u * u - sigma_u
        f_v = v_t - D_v * (v_xx + v_yy) + rho_v * f - sigma_v

        return outputs, f_u, f_v


In [54]:
type(pinn.trainable_variables)

tuple

(<tf.Variable 'D_u:0' shape=(1,) dtype=float64, numpy=array([0.99922592])>,
 <tf.Variable 'D_v:0' shape=(1,) dtype=float64, numpy=array([0.9991385])>,
 <tf.Variable 'sigma_u:0' shape=(1,) dtype=float64, numpy=array([0.99930361])>,
 <tf.Variable 'sigma_v:0' shape=(1,) dtype=float64, numpy=array([0.99930309])>,
 <tf.Variable 'mu_u:0' shape=(1,) dtype=float64, numpy=array([1.00018546])>,
 <tf.Variable 'rho_u:0' shape=(1,) dtype=float64, numpy=array([0.99981796])>,
 <tf.Variable 'rho_v:0' shape=(1,) dtype=float64, numpy=array([1.00030259])>,
 <tf.Variable 'kappa_u:0' shape=(1,) dtype=float64, numpy=array([0.99997934])>)

In [5]:
model = tu.TINN_inverse(pinn, 
                pde_loss, 
                loss = l2_norm + l_inf_norm,
                non_zero_loss=pde_non_zero, 
                #optimizer=keras.optimizers.Adam(learning_rate=0.0001), 
                alpha = 0.5,
                print_precision=".10f")

In [6]:
results = model.train(epochs=2,
                      batch_size=len(obs_X)//8,
                      X=obs_X,
                      Y=obs_Y,
                      print_interval=50,
                      stop_threshold=1e-5,
                      shuffle=True,
                      sample_losses=True, 
                      sample_parameters=True,
                      sample_regularisations=True,
                      sample_gradients=True,
                      regularise=True
                     )


Start of epoch 0


2022-10-14 18:46:56.404845: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-10-14 18:47:07.838252: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10


Training observations acc over epoch: 21.0353336334
total loss: 87.5764136743, total regularisd loss (sum of batches): 700.6113093940
obs u loss: 70.9520845484, obs v loss: 13.1892549714
pde u loss: 1.5431530750, pde v loss: 1.8919210795
pde params loss: 3.4350741545
Non-zero loss: 0.0000002004, 
lambda obs u: 1.0000000000, lambda obs v: 1.0000000000
lambda pde u: 1.0000000000, lambda pde v: 1.0000000000
lambda pde params u: 1.0000000000, lambda pde params v: 1.0000000000
D_u: 1.00035 D_v: 1.00001 sigma_u: 0.99314 sigma_v: 0.99305 
mu_u: 1.00382 rho_u: 0.99655 rho_v: 1.00368 kappa_u: 0.99491 

Time taken: 24.97s


In [7]:
model.save("/end/home/rh2310/save_tests", "model_1")

In [2]:
model2 = tu.TINN_inverse.restore("/end/home/rh2310/save_tests", "model_1")

In [18]:
pinn2 = model2.pinn
pde_residual = model2.pde_residual

In [19]:
pde_non_zero2 = Non_zero_params("ASDM", parameters=[pde_residual.D_u, pde_residual.D_v], print_precision=".10f")

In [14]:
model2 = tu.TINN_inverse(pinn2, 
                pde_residual, 
                loss = l2_norm + l_inf_norm,
                non_zero_loss=pde_residual, 
                #optimizer=keras.optimizers.Adam(learning_rate=0.0001), 
                alpha = 0.5,
                print_precision=".10f")

In [5]:
results2 = model2.train(epochs=2,
                      batch_size=len(obs_X)//8,
                      X=obs_X,
                      Y=obs_Y,
                      print_interval=50,
                      stop_threshold=1e-5,
                      shuffle=True,
                      sample_losses=True, 
                      sample_parameters=True,
                      sample_regularisations=True,
                      sample_gradients=True,
                      regularise=True
                     )


Start of epoch 0


2022-10-14 18:49:10.082413: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2022-10-14 18:49:20.475269: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10


Training observations acc over epoch: 8.5502223969
total loss: 46.0014715332, total regularisd loss (sum of batches): 950.8143121124
obs u loss: 33.0756666614, obs v loss: 1.1252231023
pde u loss: 5.4340604512, pde v loss: 6.3665213184
pde params loss: 5.9210945609
Non-zero loss: 0.0000002004, 
lambda obs u: 3.3929822281, lambda obs v: 0.5942299300
lambda pde u: 0.5024528873, lambda pde v: 0.5069706168
lambda pde params u: 0.5006600298, lambda pde params v: 0.5027043079
D_u: 0.98925 D_v: 0.99532 sigma_u: 0.99444 sigma_v: 0.99270 
mu_u: 0.99905 rho_u: 1.00326 rho_v: 0.99996 kappa_u: 0.99467 

Time taken: 23.95s
